In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
import time
import re
# Scraping through chrome driver 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [2]:
# Starting URLs
centris = "https://www.centris.ca/en/properties~for-sale?view=Thumbnail"
duproprio = "https://duproprio.com/en/search/list?search=true&is_for_sale=1&with_builders=1&parent=1&pageNumber=1&sort=-published_at"
# Path to Schromedriver
DRIVER_PATH = 'C:/webdriver/chromedriver.exe'

In [3]:
def start_driver(url:str):
    # Activate headless mode for fastest response
    options = Options()
    options.add_argument("--headless")

    # Start driver
    driver = webdriver.Chrome(executable_path=DRIVER_PATH)
    driver.get(url)
    return driver

In [4]:
def get_container_descriptions(url:str):
    driver = start_driver(url)
    container_descriptions = driver.find_elements_by_class_name("description")
    return container_descriptions

In [10]:
def get_next_page(driver):
    try:
        next_page = driver.find_element_by_xpath("//li[@class='next']")
        next_page.click()
    except:
        print("Next page button not available")

In [11]:
# Click Next button
driver = start_driver(centris)
get_next_page(driver)